# Global Precipitation Climatology Project (GPCP) -variable

In [15]:
import xarray as xr
import os
import numpy as np

In [2]:
path = '/g/data/ua8/Precipitation/GPCP/day/v1-3'
year = '2007'
folder = path + '/' + year
fileName = 'gpcp_v01r03_daily_d20070101_c20170530.nc'
path = folder + '/' + fileName

ds = xr.open_dataset(path)
ds

<xarray.Dataset>
Dimensions:      (latitude: 180, longitude: 360, time: 1, nv: 2)
Coordinates:
  * latitude     (latitude) float32 -90.0 -89.0 -88.0 -87.0 ... 87.0 88.0 89.0
  * longitude    (longitude) float32 0.0 1.0 2.0 3.0 ... 356.0 357.0 358.0 359.0
  * time         (time) datetime64[ns] 2007-01-01
Dimensions without coordinates: nv
Data variables:
    lat_bounds   (latitude, nv) float32 ...
    lon_bounds   (longitude, nv) float32 ...
    time_bounds  (time, nv) datetime64[ns] ...
    precip       (time, latitude, longitude) float32 ...
Attributes: (12/45)
    Conventions:                CF-1.6, ACDD 1.3
    title:                      Global Precipitation Climatatology Project (G...
    source:                     /data1/GPCP_CDR/GPCP_Output/1DD//bin/200701/s...
    references:                 Huffman et al. 1997, http://dx.doi.org/10.117...
    history:                    1) 2017-05-30T17:05:52Z, Dr. Jian-Jian Wang, ...
    Metadata_Conventions:       CF-1.6, Unidata Dataset Discovery v1.0, NOAA ...
    ...                         ...
    metadata_link:              gov.noaa.ncdc:XXXXX
    product_version:            v01r03
    platform:                   GOES (Geostationary Operational Environmental...
    sensor:                     Imager, AIRS > Atmospheric Infrared Sounder, ...
    spatial_resolution:         1 degree
    comment:                    Processing computer: eagle2.umd.edu

In [14]:
# path = '/g/data/ua8/Precipitation/GPCP/day/v1-3'
# folders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]

# ds_list = []
# for folder in folders:
#     folder_path = os.path.join(path, folder)
#     files = [f for f in os.listdir(folder_path) if f.endswith('.nc')]
#     for file in files:
#         ds = xr.open_dataset(os.path.join(folder_path, file))
#         ds_list.append(ds)

# ds_concat = xr.concat(ds_list, dim='time')

In [3]:
path = '/g/data/ua8/Precipitation/GPCP/day/v1-3'
years = range(2007,2018)
folders = [f for f in os.listdir(path) if (os.path.isdir(os.path.join(path, f)) and int(f) in years)]
ds_list = []
for folder in folders:
    folder_path = os.path.join(path, folder)
    files = [f for f in os.listdir(folder_path) if f.endswith('.nc')]
    for file in files:
        ds = xr.open_dataset(os.path.join(folder_path, file))
        ds_list.append(ds)

ds_concat = xr.concat(ds_list, dim='time')

In [4]:
ds_concat

<xarray.Dataset>
Dimensions:      (latitude: 180, longitude: 360, time: 4020, nv: 2)
Coordinates:
  * latitude     (latitude) float32 -90.0 -89.0 -88.0 -87.0 ... 87.0 88.0 89.0
  * longitude    (longitude) float32 0.0 1.0 2.0 3.0 ... 356.0 357.0 358.0 359.0
  * time         (time) datetime64[ns] 2010-03-30 2010-10-13 ... 2017-11-26
Dimensions without coordinates: nv
Data variables:
    lat_bounds   (time, latitude, nv) float32 -90.0 -89.0 -89.0 ... 89.0 90.0
    lon_bounds   (time, longitude, nv) float32 0.0 1.0 1.0 ... 359.0 359.0 360.0
    time_bounds  (time, nv) datetime64[ns] 2010-03-30 2010-03-31 ... 2017-11-27
    precip       (time, latitude, longitude) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
Attributes: (12/45)
    Conventions:                CF-1.6, ACDD 1.3
    title:                      Global Precipitation Climatatology Project (G...
    source:                     /data1/GPCP_CDR/GPCP_Output/1DD//bin/201003/s...
    references:                 Huffman et al. 1997, http://dx.doi.org/10.117...
    history:                    1) 2017-05-30T17:09:24Z, Dr. Jian-Jian Wang, ...
    Metadata_Conventions:       CF-1.6, Unidata Dataset Discovery v1.0, NOAA ...
    ...                         ...
    metadata_link:              gov.noaa.ncdc:XXXXX
    product_version:            v01r03
    platform:                   GOES (Geostationary Operational Environmental...
    sensor:                     Imager, AIRS > Atmospheric Infrared Sounder, ...
    spatial_resolution:         1 degree
    comment:                    Processing computer: eagle2.umd.edu

In [17]:
precip = ds_concat.precip

In [18]:
valid_range = [0, 100]
precip = precip.where((precip >= valid_range[0]) & (precip <= valid_range[1]), np.nan)

In [21]:
aWeights = np.cos(np.deg2rad(ds_concat.latitude))
precip_sMean= precip.weighted(aWeights).mean(dim=('latitude','longitude'), skipna = True)

In [23]:
# precip_sMean.plot()

In [19]:
def save_file(dataSet, folder, fileName):
    
    os.makedirs(folder, exist_ok=True)
    path = folder + '/' + fileName

    if os.path.exists(path):
        os.remove(path)    
    
    dataSet.to_netcdf(path)

In [ ]:
folder = '/g/data/k10/cb4968/data/obs/GPCP'
fileName = 'GPCP_precip_2005_2017'
dataSet = ds_concat
save_file(dataSet, folder, fileName)